<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

W tym notebooku przejdziemy całą ścieżkę - od uruchomienia modelu aż po zwrot danych w formacie JSON. Zaczynajmy! :)

### Przygotowanie środowiska

In [9]:
#%%capture

!pip install accelerate -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q


In [10]:
import gc
import torch

In [11]:
import warnings
warnings.filterwarnings("ignore") # tylko na potrzeby naszego demo ;)

### Gated models

W przypadku modeli, na które zostały nałożone zabezpieczenia w postaci kontrolowanego dostępu (gated models), wymagane będzie podanie tokena użytkownika, dostępnego na portalu HuggingFace:

* informacje dotyczące tokenów:
https://huggingface.co/docs/hub/security-tokens
* token użytkownika
https://huggingface.co/settings/tokens

W tym celu należy zainstalować bibliotekę `huggingface_hub[cli]` oraz dokonać logowania za pomocą tokena, co zostało zaprezentowane w poniższych 2 komórkach.

In [12]:
!pip install -U "huggingface_hub[cli]"

In [13]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGr

### Inicjalizacja modelu (4 bit)

In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer

device = "cuda"
model_name = "speakleash/Bielik-11B-v2.2-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             quantization_config=quantization_config
                                             )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

### Funkcja pomocnicza do generowania odpowiedzi

In [15]:
def generate(prompt: str,
             max_tokens: int = 256,
             temperature: float = 0.0,
             top_k: int = 100,
             top_p: int = 1,
             system: str = None,
             return_str: bool = False):

  messages = []

  # Przygotowanie template z promptem
  if system:
    messages.append({"role": "system",
                     "content": system})

  messages.append({"role": "user",
                   "content": prompt})

  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")
  model_inputs = input_ids

  # Przerzucenie inputu na GPU, jeśli jest dostępne
  if torch.cuda.is_available():
    model_inputs = input_ids.to(device)

  # Generowanie odpowiedzi przez model
  outputs = model.generate(
      model_inputs,
      streamer=streamer,
      max_new_tokens=max_tokens,
      do_sample=True if temperature else False,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k)

  # Wydrukowanie odpowiedzi modelu bądź zwrotka do zmiennej
  if return_str:
    decoded = tokenizer.batch_decode(outputs)
    return decoded[0]

#### Test

In [16]:
# Test z promptem systemowym i poleceniem od użytkownika
temperature = 0.1
max_tokens = 1000

generate(prompt="Hej, powiedz mi czym się zajmujesz!",
         system="Jesteś programistą baz danych, który chce rozwijać swoje kompetencje w obszarze AI",
         temperature=temperature,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



Jestem programistą baz danych, ale chciałbym rozwijać swoje umiejętności w zakresie sztucznej inteligencji (AI). Obecnie pracuję głównie z bazami danych SQL, takimi jak PostgreSQL i MySQL, oraz z systemami zarządzania bazami danych (DBMS), takimi jak Oracle Database.

Moja praca obejmuje tworzenie i utrzymywanie schematów baz danych, optymalizację zapytań SQL, zarządzanie indeksami, oraz rozwiązywanie problemów z wydajnością. Często współpracuję z analitykami danych i programistami aplikacji, aby zapewnić płynną wymianę danych między bazą danych a aplikacjami.

Teraz chciałbym rozszerzyć swoje kompetencje o AI, ponieważ widzę, że technologia ta ma ogromny potencjał w analizie i przetwarzaniu danych. Chciałbym nauczyć się, jak wykorzystywać AI do automatyzacji niektórych zadań, tworzenia modeli predykcyjnych oraz ulepszania procesów analitycznych w bazach danych.

Czy możesz mi doradzić, jakie umiejętności i technologie powinienem opanować, aby skutecznie łączyć swoją wiedzę o bazach d

# Przykład 1
Bielik świetnie nadaje się do analizy tekstu i odpowiedzi w formacie obiektu JSON. Sprawdźmy zatem jego możliwości na żywo!

Na warsztat weźmy przykładowy artykuł: https://wydarzenia.interia.pl/kraj/news-ekstremalne-warunki-niemal-w-calym-kraju-sypia-sie-alerty,nId,7763701

Najpierw musimy wydobyć tekst z artykułu! Do tego przydadzą się pythonowe biblioteki **requests** oraz **justext**!

In [17]:
# Instalacja niezbędnych bibliotek
!pip install justext -q

In [18]:
import requests
import justext

url = "https://wydarzenia.interia.pl/kraj/news-ekstremalne-warunki-niemal-w-calym-kraju-sypia-sie-alerty,nId,7763701"

def get_article_from_url(url: str):
  # Przygotowanie zmiennej do zbierania fragmentów artykułu ze strony internetowej
  article_parts = []

  # Zebranie odpowiedzi w HTML
  response = requests.get(url)

  # Przetworzenie odpowiedzi przy użyciu biblioteki justext
  paragraphs = justext.justext(response.content, justext.get_stoplist("Polish"))
  for paragraph in paragraphs:
    if not paragraph.is_boilerplate:
      article_parts.append(paragraph.text)

  return " ".join(article_parts)

In [19]:
# Test działania funkcji
get_article_from_url(url)

'Ekstremalne warunki niemal w całym kraju. Sypią się alerty Pogoda w najbliższym czasie nie da odetchnąć. Niedziela będzie kolejnym dniem z tropikalnymi upałami. W wielu miejscach termometry wskażą powyżej 30 stopni Celsjusza. W związku z ekstremalnymi warunkami IMGW wydał ostrzeżenia dla ponad połowy kraju. W lasach obowiązuje bardzo wysokie zagrożenie pożarowe. Fala upałów w Polsce. IMGW wydał ostrzeżenia/Zbyszek Kaczmarek/Agencja FORUM Reklama W niedzielę z nieba znów poleje się żar. Niemal wszędzie notowane będą ponad 30-stopniowe upały. W związku z ekstremalnymi warunkami IMGW wydał ostrzeżenia pierwszego i drugiego stopnia dla połowy kraju. Fala upałów w Polsce. IMGW wydał ostrzeżenia dla połowy kraju W ciągu dnia na wschodzie i południu Polski niebo będzie bezchmurne. Na pozostałym obszarze może wystąpić przelotny deszcz, a lokalnie - także burze. Wiatr, szczególnie nad morzem będzie dość silny. Porywy mogą sięgać do 65 km/h. Reklama Będzie upalnie. Niemal w całej Polsce słupki 

In [20]:
# Pobranie treści artykułu do zmiennej
article = get_article_from_url(url)

# Przygotowanie promptów
system = 'Jesteś ekspertem od ekstrakcji danych z tekstów i strukturyzowania ich do formatu obiektów JSON. Przeanalizuj otrzymany tekst i zwróć odpowiedź z formacie JSON z istotnymi według Ciebie kluczami.'
prompt = f"Tekst do analizy: {article}"

# Zwiększenie limitu tokenów i obniżenie temperatury (model mniej improwizuje)
temperature = 0.0
max_tokens = 1000

# Generowanie odpowiedzi Bielika!
generate(prompt=prompt,
         system=system,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.




Na podstawie podanego tekstu, oto propozycja struktury JSON z kluczami, które uważam za istotne:

```json
{
 "temperatura": {
   "zakres": "30-34°C",
   "miasta": ["Lublin", "Rzeszów", "Warszawa"]
 },
 "ostrzeżenia": {
   "stopnie": ["pierwszy", "drugi"],
   "województwa": [
     "drugiego stopnia": ["warmińsko-mazurskie", "podlaskie", "mazowieckie", "świętokrzyskie", "łódzkie", "opolskie", "śląskie", "lubelskie", "podkarpackie", "małopolskie", "kujawsko-pomorskie"],
     "pierwszego stopnia": ["małopolskie", "lubelskie", "podkarpackie"]
   ]
 },
 "zagrożenie_pożarowe": {
   "poziom": "ekstremalne, bardzo wysokie oraz wysokie",
   "komunikat": "Gorące, suche dni sprzyjają wzrostowi zagrożenia pożarowego. W związku z tym zachowajmy ostrożność planując wycieczki na łono natury."
 },
 "prognoza": {
   "do_kiedy": "do końca przyszłego tygodnia",
   "początek_września": "niewielkie ochłodzenie"
 },
 "pogoda": {
   "burze": true,
   "wiatr": {
     "nad_morzem": "dość silny",
     "porywy"

# Przykład 2

Wykorzystamy ten sam artykuł, ale damy modelowi nieco większą swobodę twórczą w prompcie!

In [21]:
article = get_article_from_url(url)

system = 'Jesteś ekspertem od ekstrakcji danych z artykułów i strukturyzowania ich do formatu obiektów JSON. Przeanalizuj otrzymany tekst i zwróć odpowiedź z formacie JSON.'
prompt = f'Tekst do analizy: {article}'
max_tokens = 1000

generate(prompt=prompt,
         system=system,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.




Oto strukturyzowane dane w formacie JSON:

```json
{
 "temperature": {
   "max": "34°C",
   "min": "30°C"
 },
 "regions": [
   "centralna",
   "południowa",
   "wschodnia"
 ],
 "wojewodztwa": [
   "warmińsko-mazurskie",
   "podlaskie",
   "mazowieckie",
   "świętokrzyskie",
   "łódzkie",
   "opolskie",
   "śląskie",
   "lubelskie",
   "podkarpackie",
   "małopolskie",
   "kujawsko-pomorskie"
 ],
 "ostrzezenia": {
   "stopien1": [
     "województwo małopolskie",
     "województwo lubelskie",
     "województwo podkarpackie"
   ],
   "stopien2": [
     "centralna",
     "południowa",
     "wschodnia"
   ],
   "susza": "36 alertów o suszy hydrologicznej, w większości dla dorzecza Wisły"
 },
 "zagrozenie_pozarowe": "ekstremalne, bardzo wysokie oraz wysokie ryzyko pożaru",
 "czas_trwania": "do końca przyszłego tygodnia",
   "poczatek_wrzesnia": "niewielkie ochłodzenie"
}
```

Ten format JSON zawiera kluczowe informacje z tekstu, takie jak maksymalne i minimalne temperatury, regiony i wojew

# Przykład 3

Dla odmiany poprosimy Bielika o zwrócenie informacji tylko dla zadanych przez nas zagadnień (bądź kluczy mówiąc językiem obiektów JSON)...

In [22]:
article = get_article_from_url(url)

system = 'Jesteś ekspertem od ekstrakcji danych z artykułów i strukturyzowania ich do formatu obiektów JSON. Przeanalizuj otrzymany tekst i zwróć go w formacie JSON. Klucze, które powinien zawierać obiekt JSON to: dzień_tygodnia, data, temperatura, zagrożenia, prognoza_na_kolejne_dni.'
prompt = f"Tekst do analizy: {article}"
max_tokens = 1000

generate(prompt=prompt,
         system=system,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.




Przeanalizowałem tekst i oto obiekt JSON zgodny z Twoim wymaganiem:

```json
{
 "dzień_tygodnia": "niedziela",
 "data": "2023-08-20",
 "temperatura": {
   "maksymalna": "34°C",
   "lokalizacje": ["Lublin", "Rzeszów", "Warszawa"]
 },
 "zagrożenia": {
   "pożarowe": "ekstremalne, bardzo wysokie oraz wysokie",
   "susza": "36 alertów o suszy hydrologicznej"
 },
 "prognoza_na_kolejne_dni": "30-stopniowe upały potrwają do końca przyszłego tygodnia, początek września ma przynieść niewielkie ochłodzenie"
}
```

Wyjaśnienie:

1. Dzień tygodnia: "niedziela"
2. Data: Chociaż nie jest podana dokładna data, można założyć, że jest to aktualny dzień (w tym przypadku 20 sierpnia 2023).
3. Temperatura:
  - Maksymalna: 34°C
  - Lokalizacje: Lublin, Rzeszów, Warszawa
4. Zagrożenia:
  - Pożarowe: Ekstremalne, bardzo wysokie oraz wysokie ryzyko pożaru
  - Susza: 36 alertów o suszy hydrologicznej
5. Prognoza na kolejne dni: 30-stopniowe upały potrwają do końca przyszłego tygodnia, początek września ma pr

## Podsumowanie
Jak widać, ekstrakcja danych do formatu JSON z tekstu jest dziecinnie prosta dla Bielika. Wystarczy krótki, prosty prompt, darmowe GPU na Colabie i voila - dane w JSONie gotowe!

Zapraszam do sprawdzenia pozostałych notebooków i tutoriali przygotowanych przez naszą społeczność!